In [1]:
#date handling
import datetime
import time

to = datetime.datetime.now()
ye = datetime.datetime.now() - datetime.timedelta(days=1)
ye1 = datetime.datetime.now() - datetime.timedelta(days=2)
ye2 = datetime.datetime.now() - datetime.timedelta(days=3)

tos = to.strftime("%Y-%m-%d")
toHs = to.strftime("%Y-%m-%d %H:%M")

yes = ye.strftime("%Y-%m-%d")
ye1s = ye1.strftime("%Y-%m-%d")
ye2s = ye2.strftime("%Y-%m-%d")
x = time.time()
x = int(x)
x48 = x - 31536000
x48 = x48*1000

print x

1496134439


In [2]:
#get sitmaps URLs 
import pymongo
import pandas as pd
pd.set_option('display.max_colwidth', -1)

con = pymongo.MongoClient(['162.21.24.111:27017'])
collsitemap = con.sitemap.newssitemap2
sitemapsdata = collsitemap.find({'unixCrawlat':{'$gte':x48}})
dfsitemap = pd.DataFrame(list(sitemapsdata))
dfsitemap = dfsitemap.drop("keywords",1)
dfsitemap = dfsitemap.drop("publication_date",1)
dfsitemap = dfsitemap.drop("unixCrawlat",1)
dfsitemap = dfsitemap.drop("crawlat",1)
dfsitemap = dfsitemap.drop("_id",1)
#dfsitemap['articleid2'] = dfsitemap['url'].str.replace(".*-id","").str.replace("[^0-9]","");
#dfsitemap = dfsitemap.drop("url",1)
#dfsitemap

In [3]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '/filer/wio/AnalyticsAPI-450d0d42f74e.json'
#KEY_FILE_LOCATION = '/Users/wio/AnalyticsAPI-450d0d42f74e.json'
VIEW_ID = '125690929'

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)
  analytics = build('analytics', 'v4', credentials=credentials)
  return analytics

def get_report(analytics):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '2daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:sessions'}],
          'pageSize': 50000,
          'dimensionFilterClauses': [
              {
                    'filters': [
                          {
                          'dimensionName': 'ga:landingPagePath',
                          'operator': 'REGEXP',
                          'expressions': [
                            '-id'
                          ]}
                    ]
              }
          ],
          'orderBys':[{
                  'fieldName': 'ga:sessions',
                  'orderType': 'VALUE',
                  'sortOrder': 'DESCENDING',
                    }],
          'dimensions': [{'name': 'ga:landingPagePath'}],
          'samplingLevel': 'LARGE'
        }]
      }
  ).execute()

def print_response(response):
  """Parses and prints the Analytics Reporting API V4 response"""

  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])

    global dfanalytics
    dfanalytics = pd.DataFrame(list(rows))
    
def main():

  analytics = initialize_analyticsreporting()
  response = get_report(analytics)
  print_response(response)

if __name__ == '__main__':
  main()

In [4]:
# bad bad bad 
tags = dfanalytics['dimensions'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'landingpage')
dfanalytics = pd.concat([dfanalytics[:], tags[:]], axis=1)  

tags = dfanalytics['metrics'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'metrics2')
dfanalytics = pd.concat([dfanalytics[:], tags[:]], axis=1) 

tags = dfanalytics['metrics2'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'metrics3')
dfanalytics = pd.concat([dfanalytics[:], tags[:]], axis=1) 

tags = dfanalytics['metrics3'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'metrics4')
dfanalytics = pd.concat([dfanalytics[:], tags[:]], axis=1)

In [5]:
dfanalytics['organicsession'] = pd.to_numeric(dfanalytics['metrics4'])
dfanalytics = dfanalytics.drop("dimensions",1)
dfanalytics = dfanalytics.drop("metrics",1)
dfanalytics = dfanalytics.drop("metrics2",1)
dfanalytics = dfanalytics.drop("metrics3",1)
dfanalytics['articleidlandingpage'] = dfanalytics['landingpage'].str.replace(".*-id","").str.replace("html.*","").str.replace("[^0-9]","");
dfanalytics = dfanalytics.drop("landingpage",1)

In [6]:
dfanalytics = pd.pivot_table(dfanalytics,index=['articleidlandingpage'], values=['organicsession'], aggfunc='sum')
dfanalytics.fillna(0, inplace=True)
dfanalytics.reset_index(inplace=True)
#dfanalytics

In [7]:
mergeddf = pd.merge(dfsitemap, dfanalytics, left_on='articleid', right_on='articleidlandingpage')
mergeddf = mergeddf.drop("articleidlandingpage",1)
#mergeddf

In [8]:
for index, row in mergeddf.iterrows():
    url = row['url']
    sessions = row['organicsession']
    
    collsitemap.update({'url':''+url},{'$push': {'traffic':{'timestamp':str(x),'organic':int(sessions)}}})
#    collsitemap.find_one_and_update({'url':''+url},{'$set': {'last4':'0,0,30,80'}})
#    collsitemap.find_one_and_update({'url':''+url},{'$set': {'insitemap':1}})
  
    last4 = row['last4']
    sumintval = 0
    for val in last4.split(','):
        sumintval = sumintval + int(val)
    
    try:
        avrintval = (sumintval * 1.0) / 4
        diff = (sessions * 1.0)-avrintval
        prozent = (diff / (sessions * 1.0))*100
    except: 
        prozent = 100

#    print "sessions " + str(sessions)
#    print "avrintval " + str(avrintval)
#    print "(sessions-avrintval) " + str(sessions-avrintval)
#    print "url " + str(url)
        
    last3 = last4.split(',', 1)[1]
    newlast4 = last3 + "," + str(sessions)   
    collsitemap.update({'url':''+url},{'$set': {'last4':''+newlast4}})
   
    if < 10:
        collsitemap.update({'url':''+url},{'$set': {'insitemap':0}})
        collsitemap.update({'url':''+url},{'$set': {'firstdropat':str(x)}})
    else:
        collsitemap.update({'url':''+url},{'$set': {'insitemap':1}})
        

  

/Users/wio/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/Users/wio/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/Users/wio/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:31: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/Users/wio/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:32: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/Users/wio/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:34: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
